In [1]:
import nltk
import pandas as pd
import sqlite3
from nltk.corpus import stopwords
from nltk import FreqDist
from collections import Counter
from collections import OrderedDict

In [2]:
con=sqlite3.connect("/Users/xiaoweichen/Kaggle/ClintonEmails/data/database.sqlite")

In [3]:
query_text="SELECT ExtractedBodyText FROM Emails AS e JOIN Persons as p ON e.SenderPersonId=p.Id"
df=pd.read_sql_query(query_text,con)

In [4]:
# Concatenate all email text sent by the same person
text_by_sender=df.groupby("Name")['ExtractedBodyText'].apply(lambda x: " ".join(x))

KeyError: 'Name'

In [5]:
def cleanup_email(raw_email_text,filter_set):
    email_text_lower=raw_email_text.lower()
    email_words=nltk.word_tokenize(email_text_lower)
    clean_email_words=[word for word in email_words if word not in filter_set]
    clean_email_text=[]
    for clean_email_word in clean_email_words:
        if not len(clean_email_word)==0:
            clean_email_text.append(clean_email_word)
    return (clean_email_text)

In [6]:
# Remove stopwords from email text
filter_set=set(stopwords.words("English"))
filter_set.update([",",".","?","!","(",")",":",";","-","'s","...","''","u","thx","fyi","pm","<",">","@","\n","un",\
                   "cc","n't","``","--","'ve","[","]","#"])
clean_text_by_sender=text_by_sender.apply(lambda x:cleanup_email(x,filter_set))

NameError: name 'text_by_sender' is not defined

In [21]:
def top_5_nouns(raw_email_text):
    pos_tags=nltk.pos_tag(raw_email_text)
    nouns=[word for (word,tag) in pos_tags if tag=="NN"]
    return [noun for (noun,freq) in FreqDist(nouns).most_common(5)]

In [ ]:
clean_text_by_sender.apply(top_5_nouns)

In [20]:
word_and_pos=nltk.pos_tag(clean_text_by_sender[20])
freq_noun=[]
# Only consider the words that are (1) noun (2) length>8
for word,category in word_and_pos:
    if category=="NN" and len(word)>8:
         freq_noun.append(word)

fdist=FreqDist(freq_noun)
fdist.most_common(10)

[(u'government', 26),
 (u'committee', 22),
 (u'settlement', 20),
 (u'community', 10),
 (u'authority', 8),
 (u'articulate', 6),
 (u'management', 6),
 (u'accountability', 6),
 (u'reconstruction', 4),
 (u'expertise', 4)]